# Encoding the Molecular Hamiltonian 

This notebook shows how to use a Fermion-Qubit encoding to encode a second quantised Molecular hamiltonian.

$$H = \sum_{i,j} h_{ij}a^{\dagger}_i a_j + \sum_{i,j,k,l} h_{ijkl}a^{\dagger}_i a^{\dagger}_j a_k a_l $$

## Simple Useage

Let's first get the coefficients for our second quantised hamiltonian.

For the time being we can use randomly generated ones.

In [1]:
import numpy as np
constant_energy = 0
one_e_coeffs = np.random.random((3,3))
two_e_coeffs = np.random.random((3,3,3,3))


Rather than figure out how many modes we need for an encoding, we can create one of the right size by passing in our coefficients.

In [2]:
from ferrmion import TernaryTree
tt = TernaryTree.from_hamiltonian_coefficients((one_e_coeffs, two_e_coeffs))

To encode a hamiltonian, we need both a mapping from fermonic operators to qubits, and an enumeration scheme which accounts for the degrees of freedom when labeling modes and qubits.

For the mapping, we'll use JordanWigner and for the enumeration scheme, we can use the default one.

Note we can [optimise our encoded hamiltonian](https://ferrmion.readthedocs.io/en/latest/notebooks/pauli_weight.html) by getting more clever with these.

In [3]:
jw = tt.JW()
jw.enumeration_scheme = jw.default_enumeration_scheme()
jw.enumeration_scheme

{'': (0, 0), 'z': (1, 1), 'zz': (2, 2)}

In [4]:
from ferrmion.hamiltonians import molecular_hamiltonian
hamiltonian = molecular_hamiltonian(encoding=jw, one_e_coeffs=one_e_coeffs, two_e_coeffs=two_e_coeffs, constant_energy=constant_energy)
hamiltonian

{'IXY': 0.15036891389244852j,
 'IZI': (-0.5012280227242117+0j),
 'YYI': (0.5082849597043216+0j),
 'XZY': -0.03501799903661859j,
 'IZZ': (0.10053982865885952+0j),
 'ZXY': -0.03816042512475579j,
 'YZX': 0.035017999036618605j,
 'ZIZ': (0.19121676507350888+0j),
 'IYX': -0.1503689138924485j,
 'XIY': -0.0409586847333958j,
 'ZXX': (-0.012937680249976907+0j),
 'III': (1.5620464852154945+0j),
 'YIX': 0.0409586847333958j,
 'XXI': (0.5082849597043217+0j),
 'IIZ': (-0.6710838948201259+0j),
 'XZX': (0.2514083097879107+0j),
 'XYZ': 0.030123493021629116j,
 'YXZ': -0.03012349302162913j,
 'YXI': -0.06304932860804259j,
 'XIX': (0.0681474151155498+0j),
 'YIY': (0.06814741511554978+0j),
 'XXZ': (-0.15614707478582524+0j),
 'YYZ': (-0.15614707478582524+0j),
 'ZYX': 0.03816042512475577j,
 'ZZI': (0.05925711950837689+0j),
 'ZYY': (-0.012937680249976893+0j),
 'ZII': (-0.7407482809119019+0j),
 'YZY': (0.25140830978791057+0j),
 'IYY': (0.142290820137206+0j),
 'XYI': 0.06304932860804265j,
 'IXX': (0.1422908201372

## Hamiltonian Templates

Sometimes it can be useful to find out how an encoding behaves without providing coefficients. This lets us see which terms of the second quantised hamiltonian contribute to which pauli terms of the qubit Hamiltonian.

### Creating a Template

In [5]:
from ferrmion.hamiltonians import molecular_hamiltonian_template

The only information we need to provide is a set of XZ-encoded vectors (this is how ferrmion manipulates encodings internally) and imaginary factors for these vectors. 

In [6]:
ipowers, symplectics = jw._build_symplectic_matrix()

In [7]:
ipowers

array([0, 1, 0, 1, 0, 1], dtype=uint8)

In [8]:
np.array(symplectics, dtype=int)

array([[1, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0],
       [0, 1, 0, 1, 0, 0],
       [0, 1, 0, 1, 1, 0],
       [0, 0, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1]])

In [9]:
template = molecular_hamiltonian_template(ipowers, symplectics, physicist_notation=True)
template

{'XZY': {(0, 2, 0, 0): 0j,
  (2, 0, 0, 0): 0j,
  (0, 2, 1, 1): 0j,
  (2, 0, 1, 1): 0j,
  (0, 0, 0, 2): 0j,
  (0, 1, 1, 2): 0.125j,
  (1, 1, 2, 0): 0j,
  (1, 2, 0, 1): -0.125j,
  (0, 2): 0.25j,
  (0, 2, 2, 2): 0j,
  (0, 0, 2, 0): 0j,
  (1, 1, 0, 2): 0j,
  (1, 0, 2, 1): 0.125j,
  (2, 2, 0, 2): 0j,
  (2, 0, 2, 2): 0j,
  (2, 1, 0, 1): 0.125j,
  (2, 1, 1, 0): -0.125j,
  (1, 2, 1, 0): 0.125j,
  (2, 2, 2, 0): 0j,
  (1, 0, 1, 2): -0.125j,
  (2, 0): -0.25j,
  (0, 1, 2, 1): -0.125j},
 'YIY': {(0, 1, 2, 1): (0.125+0j),
  (0, 1, 1, 2): (-0.125+0j),
  (1, 0, 1, 2): (0.125+0j),
  (1, 1, 2, 0): 0j,
  (1, 2, 1, 0): (0.125+0j),
  (0, 2, 1, 1): 0j,
  (2, 1, 0, 1): (0.125+0j),
  (1, 2, 0, 1): (-0.125+0j),
  (2, 0, 1, 1): 0j,
  (2, 1, 1, 0): (-0.125+0j),
  (1, 0, 2, 1): (-0.125+0j),
  (1, 1, 0, 2): 0j},
 'XXI': {(0, 0, 0, 1): 0j,
  (1, 1, 0, 1): 0j,
  (2, 1, 0, 2): (0.125+0j),
  (2, 2, 0, 1): 0j,
  (1, 0, 1, 1): 0j,
  (0, 2, 2, 1): (0.125+0j),
  (1, 0, 0, 0): 0j,
  (1, 2, 0, 2): (-0.125+0j),
  (2, 0, 2, 1

### Filling a Template

So we can now see which modes contribute to each Pauli operator.

We can now fill out our template with the coefficients we used earlier.

This time we only need to provide a mapping from fermionic modes to majorana operators. Again we can use the default.

In [10]:
jw.default_mode_op_map

array([0, 1, 2], dtype=uint64)

In [11]:
from ferrmion.hamiltonians import fill_template
filled_template = fill_template(template,constant_energy=constant_energy, one_e_coeffs=one_e_coeffs, two_e_coeffs=two_e_coeffs, mode_op_map=jw.default_mode_op_map)
filled_template

{'IXX': (0.142290820137206+0j),
 'XXZ': (-0.15614707478582524+0j),
 'IYY': (0.142290820137206+0j),
 'YYI': (0.5082849597043216+0j),
 'YZX': 0.03501799903661862j,
 'IIZ': (-0.6710838948201259+0j),
 'YZY': (0.2514083097879106+0j),
 'XXI': (0.5082849597043216+0j),
 'XYZ': 0.030123493021629102j,
 'III': (1.5620464852154943+0j),
 'ZYY': (-0.012937680249976893+0j),
 'YYZ': (-0.15614707478582524+0j),
 'IZZ': (0.10053982865885952+0j),
 'YXZ': -0.030123493021629116j,
 'XYI': 0.06304932860804263j,
 'XZY': -0.035017999036618605j,
 'ZZI': (0.05925711950837689+0j),
 'IXY': 0.15036891389244858j,
 'IYX': -0.1503689138924485j,
 'YXI': -0.06304932860804263j,
 'IZI': (-0.5012280227242119+0j),
 'ZII': (-0.7407482809119018+0j),
 'XIX': (0.06814741511554977+0j),
 'XIY': -0.0409586847333958j,
 'YIY': (0.06814741511554977+0j),
 'ZYX': 0.03816042512475579j,
 'XZX': (0.2514083097879106+0j),
 'ZXX': (-0.012937680249976866+0j),
 'YIX': 0.04095868473339581j,
 'ZIZ': (0.19121676507350888+0j),
 'ZXY': -0.0381604251

Let' check to see that we have the same hamiltonian in each.

In [12]:
assert hamiltonian.keys() == filled_template.keys()
for k in hamiltonian.keys():
    # There is a little numerical instability so 
    # some values are different by 1e-18 or so!
    assert np.isclose(hamiltonian[k], filled_template[k])